In [ ]:
import numpy as np
import random
import pdb
from collections import defaultdict
from IPython.display import clear_output


In [ ]:
# keeping it super simple, board will just be a 3x3 tuple of tuples
# action will be (row, col, player), e.g. (0, 1, 'X')
# makes it immutable/hashable, can just use board as key to q-score dict
# more sophisticated would be, put everything in a class, hash using string (or tuple) representation

BOARD_SIZE = 3
INIT_BOARD = ((' ',' ',' '),(' ',' ',' '),(' ',' ',' '),)

In [ ]:
def print_board(board):
    """string representation of board"""
    retval = ''
    for i, row in enumerate(board):
        if i:
            retval += "===========\n"
        retval += " %s\n" % " | ".join(row)
    retval += "\n"
    return retval

print(print_board(INIT_BOARD))


In [ ]:
def possible_moves(board, player='X'):
    retlist = []
    for i, row in enumerate(board):
        for j, col in enumerate(row):
            if board[i][j] == ' ':
                retlist.append((i,j, player))
    return retlist

def make_move(board, move):
    """given board and move tuple, return updated board"""
    i, j, player = move
    if i > len(board) or j > len(board):
        raise(Exception("make_move: bad square coords %d, %d" % (i,j)))
    elif board[i][j] != ' ':
        raise(Exception("make_move: move to non-empty square"))
    else:
        return tuple(row if r != i \
                     else tuple(player if c==j else square for c, square in enumerate(row)) \
                     for r, row in enumerate(board))

print(print_board(INIT_BOARD))

print("Possible_moves")
print(possible_moves(INIT_BOARD))
print()

for b in [make_move(INIT_BOARD, move) for move in possible_moves(INIT_BOARD, player='X')]:
    print(print_board(b))
    print("")

In [ ]:
# determine winner

def is_winner(board, player='X'):
    b_a = np.array(board)
    if any(all(b_a[i, :] == player) for i in range(BOARD_SIZE)):
        return True
    if any(all(b_a[:, j] == player) for j in range(BOARD_SIZE)):
        return True
    if all(np.diagonal(b_a) == player):
        return True
    if all(np.diagonal(np.fliplr(b_a)) == player):
        return True
    
    return False


[is_winner(x) for x in [((' ', ' ', ' '),(' ', ' ', ' '),(' ', ' ', ' ')),
                        (('X', 'X', 'X'),(' ', ' ', ' '),(' ', ' ', ' ')),
                        ((' ', ' ', ' '),('X', 'X', 'X'),(' ', ' ', ' ')),
                        ((' ', ' ', ' '),(' ', ' ', ' '),('X', 'X', 'X')),
                        (('X', ' ', ' '),('X', ' ', ' '),('X', ' ', ' ')),
                        ((' ', 'X', ' '),(' ', 'X', ' '),(' ', 'X', ' ')),
                        ((' ', ' ', 'X'),(' ', ' ', 'X'),(' ', ' ', 'X')),
                        (('X', ' ', ' '),(' ', 'X', ' '),(' ', ' ', 'X')),
                        ((' ', ' ', 'X'),(' ', 'X', ' '),('X', ' ', ' ')),]]



In [ ]:
# dict to hold q scores - default to 0.5 (probability X wins)
Q_scores = defaultdict(lambda: 0.5)


In [ ]:
def select_move(Q_scores, board, player, exploration_rate, verbose=False):
    """select best scoring state-action pair, 
    if more than one have best score pick random from best"""
    
    moves = possible_moves(board, player)
    # just choose a random move some % of time specified by exploration rate
    if random.uniform(0,1) < exploration_rate:
        # set all scores to 0.5
        scores = [0.5 for b in moves]
        if verbose:
            print("random exploration")
    else:
        # look up scores
        scores = [Q_scores[(board, action)] for action in moves]
        
    if verbose:
        for i, s in enumerate(scores):
            print(s, moves[i])

    # if player is X, choose highest prob of X winning else lowest prob of X winning
    best_score = max(scores) if player == 'X' else min(scores)
    # get all scores matching best
    best_moves = [moves[i] for i, score in enumerate(scores) if score == best_score]
    # pick one
    return random.choice(best_moves)


In [ ]:
# play a bunch of games computer v. computer and update Q table
NUM_GAMES = 999999
learning_rate = 0.4
discount_rate = 0.05
exploration_rate = 0.05
verbose = 0

def play_game(Q_scores,
              board_size=BOARD_SIZE,
              exploration_rate=exploration_rate,
              verbose=verbose):

    board = INIT_BOARD
    player = 'X'
    winner = None
    board_sequence = []

    for move in range(BOARD_SIZE * BOARD_SIZE):
        # current player moves
        action = select_move(Q_scores, board, player, exploration_rate, verbose=False)
        # store state, action tuple
        board_sequence.append((board, action))
        # update board
        board = make_move(board, action)
        if is_winner(board, player):
            winner = player
            break
            
        player = 'X' if player == 'O' else 'O'

    if verbose:
        for i, (b, m) in enumerate(board_sequence):
            print("Move %d" % i)
            print(print_board(b), m)

    if winner is None:
        print("Draw")
    else:
        print("%s wins" % winner)
        
    return(winner, board_sequence)        

for _ in range(NUM_GAMES):
    winner, board_sequence = play_game(Q_scores)
    # update Q_scores based on who won
    reward = 1 if winner=='X' else 0 if winner=='O' else 0.5
    for t in reversed(board_sequence):
        Q_scores[t] = Q_scores[t] + (reward - Q_scores[t]) * learning_rate
        reward = reward * (1-discount_rate)
    

In [ ]:
# check out a few q-scores
s, a = ((' ', ' ', ' '),(' ', ' ', ' '),(' ', ' ', ' ')), (2, 0, 'X')
print("%f\n%s"% (Q_scores[(s, a)], print_board(make_move(s, a))))

s, a = ((' ', ' ', ' '),(' ', ' ', ' '),('X', ' ', ' ')), (0, 1, 'O')
print("%f\n%s"% (Q_scores[(s, a)], print_board(make_move(s, a))))

s, a = ((' ', 'O', ' '),(' ', ' ', ' '),('X', ' ', ' ')), (0, 0, 'X')
print("%f\n%s"% (Q_scores[(s, a)], print_board(make_move(s, a))))

s, a = (('X', 'O', ' '),(' ', ' ', ' '),('X', ' ', ' ')), (1, 0, 'O')
print("%f\n%s"% (Q_scores[(s, a)], print_board(make_move(s, a))))

s, a = (('X', 'O', ' '),('O', ' ', ' '),('X', ' ', ' ')), (2, 2, 'X')
print("%f\n%s"% (Q_scores[(s, a)], print_board(make_move(s, a))))

s, a = (('X', 'O', ' '),('O', ' ', ' '),('X', ' ', 'X')), (1, 1, 'O')
print("%f\n%s"% (Q_scores[(s, a)], print_board(make_move(s, a))))

s, a = (('X', 'O', ' '),('O', 'O', ' '),('X', ' ', 'X')), (2, 1, 'X')
print("%f\n%s"% (Q_scores[(s, a)], print_board(make_move(s, a))))



In [ ]:

z[(90)].append('sdf')

In [ ]:
import itertools

z = defaultdict(list)

for key, v in Q_scores.items():
    s, a = key
    # flatten
    s = make_move(s, a)
    s = list(itertools.chain.from_iterable(s))
    # map s to float
    s = tuple(0 if i==' ' else 1 if i=='X' else -1 for i in s)
    templist = z[s]
    templist.append(v)
    z[s]=templist

z2=dict()
for k, l in z.items():
    z2[k]=sum(l)/len(l)

import pandas as pd



In [ ]:
V = pd.DataFrame(z2.keys())
V['val']=z2.values()
V.to_csv('V.csv')


In [ ]:
# play (and learn) interactively
verbose = True
exploration_rate = 0.05
def get_move(prompt):
    move = None
    while move not in ['1', '2', '3']: 
        print(prompt)
        move = input()
    return int(move)-1

def play_again():
    print('Play again? (y or n)')
    return input().lower().startswith('y')

while True:
    board = INIT_BOARD
    winner = None
    board_sequence = []
    # X is human and goes first
    player = 'X'
    
    clear_output()
    print(print_board(board))
    
    for _ in range(BOARD_SIZE * BOARD_SIZE):
        if player == 'X':
            while True:
                row = get_move("Enter row: ")
                col = get_move("Enter column: ")
                try:
                    board = make_move(board, row, col, player)
                except Exception as e:
                    print(str(e))
                    continue
                break
        else:
            board = select_move(Q_scores, board, player, exploration_rate=exploration_rate, verbose=verbose)
        board_sequence.append(board)

        clear_output()
        print(print_board(board))
        
        if is_winner(board, player):
            winner = player
            break
        player = 'X' if player == 'O' else 'O'

    if winner is None:
        print("Draw")
    else:
        print("%s wins!" % winner)

    # update Q_scores based on who won
    reward = 1 if winner=='X' else -1 if winner=='O' else 0
    for b in reversed(board_sequence):

        Q_scores[b] = Q_scores[b] + (reward - Q_scores[b]) * learning_rate
        if verbose:
            print(reward)
            print("%f\n%s"% (Q_scores[b], print_board(b)))
        
        reward = reward * (1-discount_rate)
        
    if not play_again():
        break


In [ ]:
## def create_all_boards(start_boards, move, player):
    """create all possible boards starting from start_boards, with player to move"""
    
    all_boards = []

    for _ in range(BOARD_SIZE*BOARD_SIZE - move):
        
        all_boards += start_boards
        current_boards = []
        move += 1
        # set opponent
        opponent = 'O' if player == 'X' else 'X'

        print("###########################")
        print("Move %d, %d starting boards" % (move, len(start_boards)))
        print("###########################")
        print("")

        for newb in start_boards:
            # skip if last move ended game
            if is_winner(newb, opponent):
                continue
            current_boards += [make_move(newb, i, j, player) for i,j in possible_moves(newb, player)]

        # uniquify
        current_boards = list(set(current_boards))
        
        for i, newb in enumerate(current_boards):
            print("Move %d, board %d" % (move, i+1))
            print(print_board(newb))
            print("")
        start_boards = current_boards
        
        # switch player
        player = 'O' if player == 'X' else 'X'
        
    return all_boards
    
all_boards = create_all_boards([INIT_BOARD], 0, 'O')
print(len(all_boards))


In [ ]:
def change_dict(a):
    a['lsdfj'] = 1
    
z = dict()
print(z)
change_dict(z)
print(z)